# Data Cleaning

## Importing Libraries

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

## Cleaning Retraction Data

### General Cleaning

In [ ]:
retract = pd.read_csv('./total_retraction_data.csv', index_col=False)

As discussed in the "Data Collection" notebook, the querying script would have ideally run through the entire DOI column. However, I often had 404 errors or the script would stop abruptly with no error message. For this reason, I decided to break the DOI column into small sections to closely monitor the information and save the information I received frequently. Because of this, several CSV files were created, each labeled with what DOI index values were used when pulling the information. This information was then cleaned into one complete CSV file in a notebook that will not be provided. This CSV file is labeled as "total_retraction_data.csv" and was opened above. 

In [5]:
retract = retract.drop(columns=['Unnamed: 0', 'doi_check'])

In [ ]:
retract = retract.dropna(axis=0, subset=['text'])

In [ ]:
retract['retraction_binary'] = 1

In [8]:
retract.to_csv('./no_null_text_retraction_data.csv')
retract = pd.read_csv('./no_null_text_retraction_data.csv')
retract = retract.drop(columns=['Unnamed: 0'])

The dataframe was cleaned in several ways. Unnecessary columns were dropped. Any articles that had no text data were dropped. A new column named "retraction_binary" was created that was filled with the integer 1 to indicate that all of the articles in this dataframe were articles that had been retracted. Using .info method below, it is evident that the only remaining articles had full text. Once the data was cleaned, it was saved into a new CSV file for proofing.

In [9]:
retract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1553 non-null   int64  
 1   doi                1553 non-null   object 
 2   language           1553 non-null   object 
 3   year               1315 non-null   float64
 4   month              1315 non-null   float64
 5   day                1315 non-null   float64
 6   volume             1518 non-null   float64
 7   issue              1043 non-null   float64
 8   journal            1553 non-null   object 
 9   title              1553 non-null   object 
 10  page               1487 non-null   object 
 11  text               1553 non-null   object 
 12  abstract           1533 non-null   object 
 13  keywords           1553 non-null   object 
 14  publisher          1549 non-null   object 
 15  retraction_binary  1553 non-null   int64  
dtypes: float64(5), int64(2),

### Unpacking Keywords

In [10]:
ls_keywords_list = []
count = 0
for i in retract['keywords']:
    keywords_list = []
    if i == []:
        ls_keywords_list.append([])
    else:
        for j in i.split():
            keywords_list.append(j.replace("'",'').replace('[','').replace(',','').replace(']','').replace('(','').replace(')','').replace('\\n', '').replace('\\n','').lower())
        if keywords_list == ['']:
            ls_keywords_list.append([])
        else:
            ls_keywords_list.append(keywords_list)

retract['unpacked_keywords'] = ls_keywords_list

The keywords list for each article was a string. In this format, this information is not useful. Because of this, the above script was used to "unpack" each keyword and place it into a true list. These lists were then placed into a new column, "unpacked_keywords".

### Cleaning and Lemmatizing Text

In [ ]:
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

clean_text = []
clean_text_lem = []

for i in range(0, len(retract['text'])):
    ls_words = []
    ls_lem = []
    for j in tokenizer.tokenize(retract['text'][i]):
        try:
            int(j)
        except:
            if len(j) < 45:
                ls_words.append(j)
                ls_lem.append(lemmatizer.lemmatize(j))
            else:
                pass
        
    clean_text.append(' '.join(ls_words))
    clean_text_lem.append(' '.join(ls_lem))
    
    print(i)

In [ ]:
retract['clean_text'] = clean_text
retract['clean_text_lem'] = clean_text_lem

The text of each article as it was originally received from the query needed to be cleaned in several ways. There were several symbols and numerical values that needed to be removed. To remove these characters, the RegexpTokenizer was used. Additionally, several "words" were in fact URLs or strings of characters that were not intelligible (as they may have been artifacts from mathematical equations in the article). Thus, any string that was above 45 characters was ignored. Once these cleaning steps had been taken, the text was saved in two different ways: one method with no further formatting, and one method where a lemmatizer was used. The now two different text bodies were then combined back together using spaces and added to new columns in the dataframe. Text that had no further formatting was saved as "clean_text" while lemmatized text was saved as "clean_text_lem".

## Cleaning No Retraction Data

### Correcting Indexing Error During PMC Querying

In [15]:
#reading all text CSV files from PMC querying
text_one = pd.read_csv('./text__no_retraction__0_5545.csv')
text_two = pd.read_csv('./text__no_retraction__5545_6166.csv')
text_three = pd.read_csv('./text__no_retraction__6166_end.csv')

#reading all abstract CSV files from PMC querying
abstract_one = pd.read_csv('./abstract__no_retraction__0_5545.csv')
abstract_two = pd.read_csv('./abstract__no_retraction__5545_6166.csv')
abstract_three = pd.read_csv('./abstract__no_retraction__6166_end.csv')

#reading all keyword CSV files from PMC querying
keywords_one = pd.read_csv('./keywords__no_retraction__0_5545.csv')
keywords_two = pd.read_csv('./keywords__no_retraction__5545_6166.csv')
keywords_three = pd.read_csv('./keywords__no_retraction__6166_end.csv')

#reading all publisher CSV files from PMC querying
publisher_one = pd.read_csv('./publisher__no_retraction__0_5545.csv')
publisher_two = pd.read_csv('./publisher__no_retraction__5545_6166.csv')
publisher_three = pd.read_csv('./publisher__no_retraction__6166_end.csv')

#reading all DOI CSV files from PMC querying
doi_one = pd.read_csv('./doi__no_retraction__0_5545.csv')
doi_two = pd.read_csv('./doi__no_retraction__5545_6166.csv')
doi_three = pd.read_csv('./doi__no_retraction__6166_end.csv')

As discussed in the "Data Collection" notebook, the script would have ideally run through the entire DOI column. However, when using this function, I often had 404 errors or the script would stop abruptly with no error message. For this reason, I decided to break the DOI column into three different sections to closely monitor the information and save the information I received more frequently. Because of this, three different CSV files were created, each labeled with what DOI index values were used when pulling the information. These CSV files are opened in the above script.

In [16]:
text_one.tail()

,Unnamed: 0,0
5541,5541,"Winter air pollution in Ulaanbaatar, Mongolia ..."
5542,5542,Chronic infection with hepatitis C virus (HCV)...
5543,5543,California has one of the most highly engineer...
5544,5544,Thiol-dependent cathepsins are found in all li...
5545,5545,To estimate hepatitis C virus (HCV) viremic ra...


In [17]:
text_two.head()

,Unnamed: 0,0
0,0,To estimate hepatitis C virus (HCV) viremic ra...
1,1,Pollinators are crucial in almost all terrestr...
2,2,The association of melanosis coli with the dev...
3,3,"In most plant species, repetitive DNA constitu..."
4,4,"In the last decades, there has been a great in..."


In [18]:
text_two.tail()

,Unnamed: 0,0
617,617,Bread wheat (Triticum aestivum L.) is one of t...
618,618,"Metabolic syndrome (MetS), defined as a comple..."
619,619,Competitive learning techniques are being succ...
620,620,"In South East Africa, about 100,000 years ago ..."
621,621,In the text of González-Fernández [1] can be f...


In [19]:
text_three.head()

,Unnamed: 0,0
0,0,In the text of González-Fernández [1] can be f...
1,1,Infantile spasms (IS) are the defining seizure...
2,2,The arylamine N-acetyltransferases are a famil...
3,3,Several biomarkers have been proposed for ultr...
4,4,Osteoporosis is a skeletal disease characteriz...


I realized I had duplicated two query calls when completing the PMC no retraction querying for the no retraction data. Because of this, I had to drop the appropriate rows from the CSV files for each piece of information determined while querying. The rows were dropped in the script below.

In [24]:
#dropping duplicate entries from text dataframes
text_one = text_one.drop(5545, axis=0)
text_two = text_two.drop(621, axis=0)

#dropping duplicate entries from abstract dataframes
abstract_one = abstract_one.drop(5545, axis=0)
abstract_two = abstract_two.drop(621, axis=0)

#dropping duplicate entries from keywords dataframes
keywords_one = keywords_one.drop(5545, axis=0)
keywords_two = keywords_two.drop(621, axis=0)

#dropping duplicate entries from publisher dataframes
publisher_one = publisher_one.drop(5545, axis=0)
publisher_two = publisher_two.drop(621, axis=0)

In [27]:
#combining all text dataframes
text_total = pd.concat([text_one, text_two, text_three], axis =0)

#combining all DOI dataframes
doi_total = pd.concat([doi_one, doi_two, doi_three], axis =0)

#combining all abstract dataframes
abstract_total = pd.concat([abstract_one, abstract_two, abstract_three], axis =0)

#combining all keyword dataframes
keywords_total = pd.concat([keywords_one, keywords_two, keywords_three], axis =0)

#combining all publisher dataframes
publisher_total = pd.concat([publisher_one, publisher_two, publisher_three], axis =0)

Each CSV file for each piece of information from the PMC query was concatenated together by row. Once the dataframe was created for each piece of information, the dataframe was then saved as a CSV file in the script below for proofing.

In [28]:
#saving all concatenated dataframes for proofing
text_total.to_csv('./plos_only_no_retraction_text.csv')
doi_total.to_csv('./plos_only_no_retraction_doi.csv')
abstract_total.to_csv('./plos_only_no_retraction_abstract.csv')
keywords_total.to_csv('./plos_only_no_retraction_keywords.csv')
publisher_total.to_csv('./plos_only_no_retraction_publisher.csv')

In [29]:
#rereading all concatenated dataframes to continue working
text = pd.read_csv('./plos_only_no_retraction_text.csv')
abstract = pd.read_csv('./plos_only_no_retraction_abstract.csv')
keywords = pd.read_csv('./plos_only_no_retraction_keywords.csv')
publisher = pd.read_csv('./plos_only_no_retraction_publisher.csv')
doi = pd.read_csv('./plos_only_no_retraction_doi.csv')

### Combining Data from PMC Queries

In [ ]:
#pulling all text data for no retractions
ls_text = []
for i in text['0']:
    ls_text.append(i)
print(len(ls_text))

#pulling all abstract data for no retractions
ls_abstract = []
for i in abstract['0']:
    ls_abstract.append(i)
print(len(ls_abstract))

#pulling all keywords list data for no retractions
ls_keywords = []
for i in keywords['0']:
    ls_keywords.append(i)
print(len(ls_keywords))

#pulling all publisher data for no retractions
ls_publisher = []
for i in publisher['0']:
    ls_publisher.append(i)
print(len(ls_publisher))

#pulling all DOI data for no retractions
ls_doi = []
for i in doi['1']:
    ls_doi.append(i)
print(len(ls_doi))

To place all of the information from the PMC no retraction query, the information from the newly formed CSV files was pulled and placed into a list for each information piece. The PubMed no retraction CSV file was then opened. All of the lists were concatenated with the PubMed no retraction dataframe to combine all of the no retraction inforamtion into a single dataframe.

In [31]:
no_retract = pd.read_csv('./pubmed_data_second_no_retraction.csv')

In [32]:
no_retract = pd.concat([no_retract, pd.Series(ls_text), pd.Series(ls_abstract), pd.Series(ls_keywords), 
                        pd.Series(ls_publisher), pd.Series(ls_doi)], axis=1)

In [33]:
no_retract = no_retract.drop(columns=['Unnamed: 0'])

In [34]:
no_retract = no_retract.rename(columns={'0':'id', '1':'doi', '2':'language', '3':'year', '4':'month', '5':'day', 
                      '6':'volume', '7':'issue', '8':'journal', '9':'title', '10':'page', 
                      0:'text', 1:'abstract', 2:'keywords', 3:'publisher', 4:'doi_check'})

Unnecessary columns were dropped. For the columns that remain, the columns were renamed to the appropriate information based on the order in which the information was pulled during querying. The complete dataframe can be seen below.

In [36]:
no_retract['doi'][5540:5550]

5540    10.1371/journal.pone.0186821
5541    10.1371/journal.pone.0186834
5542    10.1371/journal.pone.0186898
5543    10.1371/journal.pone.0187181
5544    10.1371/journal.pone.0186869
5545    10.1371/journal.pone.0187177
5546    10.1371/journal.pone.0187079
5547    10.1371/journal.pone.0186668
5548    10.1371/journal.pone.0187131
5549    10.1371/journal.pone.0186957
Name: doi, dtype: object

In [37]:
no_retract['doi_check'][5540:5550]

5540    10.1371/journal.pone.0186821
5541    10.1371/journal.pone.0186834
5542    10.1371/journal.pone.0186898
5543    10.1371/journal.pone.0187181
5544    10.1371/journal.pone.0186869
5545    10.1371/journal.pone.0187177
5546    10.1371/journal.pone.0187079
5547    10.1371/journal.pone.0186668
5548    10.1371/journal.pone.0187131
5549    10.1371/journal.pone.0186957
Name: doi_check, dtype: object

In [39]:
no_retract.to_csv('./plos_only_no_retraction_data.csv')

The DOI values were compared between the two different DOI columns to ensure that the concatenating between the different dataframes was completed accurately. Once it was determined that there were no errors in concatenating, the dataframe was saved as a new CSV file for proofing.

### General Cleaning

In [40]:
no_retract = pd.read_csv('./plos_only_no_retraction_data.csv', index_col=False)
no_retract = no_retract.drop(columns=['Unnamed: 0', 'doi_check'])

In [41]:
print(no_retract['abstract'].isnull().sum())
print(no_retract['text'].isnull().sum())

494
399


The above script shows the number of null values in the "abstract" and "text" column. In the case that there is simply too much text in the full article to be used for NLP modeling, it may be possible to use the information in the abstract of each article.

In [ ]:
no_retract = no_retract.dropna(axis=0, subset=['text'])

Any articles that were missing text were dropped from the dataframe.

In [43]:
np.random.choice(no_retract['doi'], 50)

array(['10.1371/journal.pone.0221236', '10.1371/journal.pone.0208797',
       '10.1371/journal.pone.0145158', '10.1371/journal.pone.0184077',
       '10.1371/journal.pone.0169155', '10.1371/journal.pone.0119129',
       '10.1371/journal.pone.0175673', '10.1371/journal.pone.0226358',
       '10.1371/journal.pone.0186309', '10.1371/journal.pone.0144882',
       '10.1371/journal.pone.0217685', '10.1371/journal.pone.0168498',
       '10.1371/journal.pone.0203429', '10.1371/journal.pone.0147806',
       '10.1371/journal.pone.0210432', '10.1371/journal.pone.0201007',
       '10.1371/journal.pone.0155713', '10.1371/journal.pone.0178620',
       '10.1371/journal.pone.0135598', '10.1371/journal.pone.0178231',
       '10.1371/journal.pone.0156508', '10.1371/journal.pone.0118722',
       '10.1371/journal.pone.0179354', '10.1371/journal.pone.0221109',
       '10.1371/journal.pone.0171148', '10.1371/journal.pone.0152195',
       '10.1371/journal.pone.0131134', '10.1371/journal.pone.0216493',
      

50 DOIs of articles that were not retracted from PLOS ONE were pulled. These DOIs were used to manually check that the articles pulled during the query were in fact not retracted. While all of the articles proved to be not retracted, 5 of the articles had actually been corrected. The reasons for correction were as followed: author byline changes and fixing a citation, supporting figure appears incorrectly, authors spelled incorrectly, errors in figures, and affiliation listing error. Corrections are not the same as retractions, but may be something to look into in the future.

In [ ]:
no_retract['retraction_binary'] = 0
print(no_retract['retraction_binary'].value_counts())
no_retract.head()

In [45]:
no_retract.to_csv('./no_null_text_plos_only_no_retraction_data.csv')
no_retract = pd.read_csv('./no_null_text_plos_only_no_retraction_data.csv')
no_retract = no_retract.drop(columns=['Unnamed: 0'])

A new column named "retraction_binary" was created that was filled with the integer 0 to indicate that all of the articles in this dataframe were articles that had not been retracted. Using .info method below, it is evident that the only remaining articles had full text. Once the data was cleaned, it was saved into a new CSV file for proofing.

In [46]:
no_retract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9214 entries, 0 to 9213
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 9214 non-null   int64 
 1   doi                9214 non-null   object
 2   language           9214 non-null   object
 3   year               9214 non-null   int64 
 4   month              9214 non-null   int64 
 5   day                9214 non-null   int64 
 6   volume             9214 non-null   int64 
 7   issue              9214 non-null   int64 
 8   journal            9214 non-null   object
 9   title              9214 non-null   object
 10  page               9214 non-null   object
 11  text               9214 non-null   object
 12  abstract           9119 non-null   object
 13  keywords           9214 non-null   object
 14  publisher          9214 non-null   object
 15  retraction_binary  9214 non-null   int64 
dtypes: int64(7), object(9)
memory usage: 1.1+ 

### Unpacking Keywords

In [47]:
no_retract['keywords'].value_counts()

[]                                                                            9213
['Socioeconomic inequality', 'Antenatal care', 'Decomposition', 'Nigeria']       1
Name: keywords, dtype: int64

I went to unpack the keywords list as I had done for the retraction data. However, after completing .value_counts method for the column, I realized that only one data point out of 9214 articles actually had a keywords list. I manually checked using the same 50 random DOI values to determine that none of the articles randomly chosen actually had a keywords list. Because of this, I did not bother to unpack the keywords list.

### Cleaning and Lemmatizing Text

In [ ]:
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

clean_text = []
clean_text_lem = []

for i in range(0, len(no_retract['text'])):
    ls_words = []
    ls_lem = []
    for j in tokenizer.tokenize(no_retract['text'][i]):
        try:
            int(j)
        except:
            if len(j) < 45:
                ls_words.append(j)
                ls_lem.append(lemmatizer.lemmatize(j))
            else:
                pass
        
    clean_text.append(' '.join(ls_words))
    clean_text_lem.append(' '.join(ls_lem))
    
    print(i)

print(len(no_retract['text'][1]))
print(len(clean_text[1]))
print(len(clean_text_lem[1]))

In [ ]:
no_retract['clean_text'] = clean_text
no_retract['clean_text_lem'] = clean_text_lem
no_retract.head()

The text of each article as it was originally received from the query needed to be cleaned in several ways. There were several symbols and numerical values that needed to be removed. To remove these characters, the RegexpTokenizer was used. Additionally, several "words" were in fact URLs or strings of characters that were not intelligible (as they may have been artifacts from mathematical equations in the article). Thus, any string that was above 45 characters was ignored. Once these cleaning steps had been taken, the text was saved in two different ways: one method with no further formatting, and one method where a lemmatizer was used. The now two different text bodies were then combined back together using spaces and added to new columns in the dataframe. Text that had no further formatting was saved as "clean_text" while lemmatized text was saved as "clean_text_lem".

In [61]:
no_retract.loc[no_retract['year']==2014].index

Int64Index([ 43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
            ...
            157, 158, 159, 160, 161, 162, 163, 164, 165, 166],
           dtype='int64', length=124)

In [68]:
no_retract = no_retract.drop(range(43, 167))
no_retract['year'].value_counts()

2019    1891
2016    1833
2015    1792
2018    1788
2017    1786
Name: year, dtype: int64

During the querying process, the dates randomly chosen were used as the end point of the query. Thus, 167 articles were pulled where each date is the last possible date the article could have been published on. Because of this, there were 124 articles that were published in 2014. For consistency, these articles were dropped.

## Cleaning Combined Datasets

In [ ]:
total = pd.concat([retract, no_retract], axis=0)
total

The retraction and no retraction dataframes were concatenated together to create a dataframe that has all of the inforamtion from all of the queries completed in the "Data Collection" notebook.

In [71]:
total = total.drop(columns=['id', 'language', 'publisher'])

In [73]:
total = total.drop(columns=['page'])

In [74]:
total['year'] = total['year'].astype(str)
total['month'] = total['month'].astype(str)
total['day'] = total['day'].astype(str)
total['volume'] = total['volume'].astype(str)
total['issue'] = total['issue'].astype(str)

Unnecessary columns were dropped. The "year," "month," "day," "volume," and "issue" columns were retyped as strings so that they could be dummied in later notebooks. Once cleaned, the dataframe was saved into a new CSV file for proofing.

In [76]:
total.to_csv('./total_plos_only_data.csv')
total = pd.read_csv('./total_plos_only_data.csv')

In [77]:
total['doi'].value_counts().head(25)

10.1074/jbc.M111.329078           4
10.1371/journal.pone.0194078      2
10.1038/cr.2011.194               2
10.1074/jbc.M111.275073           2
10.1371/journal.pone.0164378      2
10.1038/cdd.2010.114              2
10.1371/journal.pone.0212021      2
10.1074/jbc.M110.175802           2
10.3390/nano4020203               2
10.1074/jbc.M808084200            2
10.1371/journal.pone.0216079      2
10.1371/journal.pone.0155697      2
10.1523/JNEUROSCI.0372-13.2013    2
10.1371/journal.pone.0146671      2
10.1074/jbc.M112.387738           2
10.1200/JCO.2017.74.7824          2
10.1155/2012/236409               2
10.1074/jbc.M709854200            2
10.1371/journal.pone.0140044      2
10.1371/journal.pone.0125542      2
10.3389/fnins.2018.00529          2
10.1371/journal.pone.0183066      1
10.1371/journal.pone.0225345      1
10.1371/journal.pone.0193981      1
10.1371/journal.pone.0166478      1
Name: doi, dtype: int64

In [78]:
total = total.drop_duplicates(subset='doi', keep='first')

During the querying process, some DOI values were repeated even though duplicates should have been dropped before pulling from the PMC database. These DOI values are unique strings that identify a specific article. Because of this, if a DOI is repeated, then the article text is repeated within the dataframe as well. Thus, these duplicates were dropped and saved to a new CSV file for proofing.

In [80]:
total.to_csv('./total_plos_only_data_no_duplicates.csv')
total = pd.read_csv('./total_plos_only_data_no_duplicates.csv')

In [81]:
total['clean_text'][950:955]

950    For the chemotherapeutic activity of pyrimidin...
951    Urinary tract infection UTI is a bacterial inf...
952    The Bible descrbies the case of a woman with h...
953                                                  NaN
954    Human betaherpesviruses 6A and 6B HHV 6A and H...
Name: clean_text, dtype: object

During the tokenizing process, one text must have been removed completely (as the text may not have had any spaces). This row was dropped as it serves no purpose to modeling.

In [84]:
total.to_csv('./total_data_plos_only_cleaned.csv')
total[1537:].to_csv('./no_retraction_data_plos_only_cleaned.csv')
total[:1537].to_csv('./retraction_data_plos_only_cleaned.csv')

Because of the way the dataframes were concatenated, the dataframe from index 0-1537 are only retracted articles while the index 1537-10643 are only non-retracted articles. Thus, the final dataframe was saved in three different ways: the complete dataframe, only retracted articles, and only non-retracted articles.